# this manuscript is to calculate ice phenology from daily lake ice thickness in CESM2-LE

In [1]:
import numpy as np # version '1.20.0'
import xarray as xr # version '0.16.2'
from tqdm import tqdm # version '4.43.0'
import matplotlib.pyplot as plt # version '3.3.2'
import matplotlib as mpl # version '3.3.2'
import cmocean # version '2.0'
import cartopy.crs as ccrs # version '0.18.0'
import cartopy.feature as cf # version '0.18.0'
import glob
import dask.array as da # version '2021.1.1'
import time
from numpy import nan
import seaborn as sns # version '0.11.0'
import pandas as pd # version 1.2.1'
import matplotlib.path as mpath

In [3]:
# setup the dask parallel computing
from dask.distributed import Client
client = Client(scheduler_file= '/.../scheduler.json') # the path to the scheduler file
client

Client Scheduler: tcp://203.247.189.224:43634 Dashboard: http://203.247.189.224:8787/status,Cluster Workers: 16 Cores: 576 Memory: 320.00 GB


# 1. define functions

## 1.1 define functions to read in nc files across all the ensemble members
The variables stored in the directories of individual ensemble members ,we use this function to read the data of all the ensemble members into one xarray dataset

In [8]:
def def_process_coords(exceptcv=[]):
    def process_coords(ds, except_coord_vars=exceptcv):
        coord_vars = []
        for v in np.array(ds.coords):
            if not v in except_coord_vars:
                coord_vars += [v]
        for v in np.array(ds.data_vars):
            if not v in except_coord_vars:
                coord_vars += [v]
        return ds.drop(coord_vars)
    return process_coords
#-----------------------------------------------------------------------------------------------------
def read_in(var, exceptcv, domain='lnd/', freq='day_1/', stream='h6', chunks=dict(time=365), ens_s=-20, ens_e=-10):
    ens_dir = "path for the home directory of the whole CESM-LE data"
    histens_names = [member.split('archive/')[1][:-1]
                     for member in sorted(glob.glob(ens_dir + "b.e21.BHIST*LE2*[!old][!tmp]/"))][10:]
    projens_names = [member.split('archive/')[1][:-1] for member in sorted(
        glob.glob(ens_dir + "b.e21.BSSP370*.f09_g17*[!old][!tmp]/"))][10:]
    hist_ncfiles = []
    proj_ncfiles = []
    for i in np.arange(ens_s, ens_e):
        hist_fnames = sorted(glob.glob(
            ens_dir + histens_names[i] + "/" + domain + "proc/tseries/" + freq + "*" + stream + var + "*"))
        proj_fnames = sorted(glob.glob(
            ens_dir + projens_names[i] + "/" + domain + "proc/tseries/" + freq + "*" + stream + var + "*"))
        hist_ncfiles.append(hist_fnames)
        proj_ncfiles.append(proj_fnames)
    ens_numbers = [members.split('LE2-')[1]
                   for members in histens_names][ens_s:ens_e]
    hist_ds = xr.open_mfdataset(hist_ncfiles,
                                chunks=chunks,
                                preprocess=def_process_coords(exceptcv),
                                combine='nested',
                                concat_dim=[[*ens_numbers], 'time'],
                                parallel=True)
    proj_ds = xr.open_mfdataset(proj_ncfiles,
                                chunks=chunks,
                                preprocess=def_process_coords(exceptcv),
                                combine='nested',
                                concat_dim=[[*ens_numbers], 'time'],
                                parallel=True)
    if freq == 'day_1/':
        hist_ds = hist_ds.isel(time=np.arange(1, hist_ds.time.shape[0]))
        proj_ds = proj_ds.isel(time=np.arange(1, proj_ds.time.shape[0]))
        hist_ds['time'] = hist_ds.time.get_index('time').shift(-1, 'D')
        proj_ds['time'] = proj_ds.time.get_index('time').shift(-1, 'D')
    if freq == 'month_1/':
        hist_ds['time'] = hist_ds.time.get_index('time').shift(-1, 'M')
        proj_ds['time'] = proj_ds.time.get_index('time').shift(-1, 'M')
    ens_ds = xr.concat((hist_ds, proj_ds), 'time')
    ens_ds = ens_ds.rename({'concat_dim': 'ensemble'})
    return ens_ds


# 2. Calculate iceduration, iceon, iceoff

## 2.1 read in daily ice thickness data from CESM2-LE

In [25]:
variables = ['LAKEICETHICK']
exceptcv = ['time', 'lat', 'lon', *variables]
ice_ds = read_in(var = '.LAKEICETHICK.', 
                 exceptcv= exceptcv, 
                 freq = 'day_1/', 
                 stream = 'h5',
                 chunks =dict(time= 50), 
                 ens_s = 0, 
                 ens_e = 90)

## 2.2 calculate ice phenology from daily lake ice thickness in CESM2-LE

In [27]:
'''
in the Northern Hemisphere, one ice cover season spread among two calendar years,
to get the right day of year for ice on data, we shift the whole time index by -232 days, so that the ice season is within one calendar year and easy to calculate the ice duraiton, iceon date, ice off date
'''
daily_ice  = ice_ds.LAKEICETHICK
daily_ice['time'] = daily_ice.time.get_index('time').shift(-232,"D")
daily_ice  = daily_ice.sel(time = slice('1850-01-01','2099-12-31'))
years = np.unique(daily_ice.time.dt.year)
dayofyear = np.unique(daily_ice.time.dt.dayofyear)
ind = pd.MultiIndex.from_product((years,dayofyear),names=('year','dayofyear'))
daily_ice_res = daily_ice.assign_coords(time = ind).unstack('time')

In [28]:
iceon_mod = xr.DataArray(nan, 
                     attrs=daily_ice.attrs,
                     name='LAKEICETHICK',
                     dims=('ensemble','lat','lon','year'), 
                     coords = {'ensemble':daily_ice.ensemble,'lat':daily_ice.lat,'lon':daily_ice.lon,'year':np.arange(1850,2100)})
iceoff_mod = xr.DataArray(nan, 
                     attrs=daily_ice.attrs,
                     name='LAKEICETHICK',
                     dims=('ensemble','lat','lon','year'), 
                     coords = {'ensemble':daily_ice.ensemble,'lat':daily_ice.lat,'lon':daily_ice.lon,'year':np.arange(1850,2100)})
iceduration_mod = xr.DataArray(nan, 
                     attrs=daily_ice.attrs,
                     name='LAKEICETHICK',
                     dims=('ensemble','lat','lon','year'), 
                     coords = {'ensemble':daily_ice.ensemble,'lat':daily_ice.lat,'lon':daily_ice.lon,'year':np.arange(1850,2100)})
daily_ice_max = xr.DataArray(nan,
                            attrs = daily_ice.attrs,
                            name='LAKEICETHICK',
                            dims = ('ensemble','lat','lon','year'),
                            coords = {'ensemble':daily_ice.ensemble,'lat':daily_ice.lat,'lon':daily_ice.lon, 'year':np.arange(1850,2100)})
ice_thick_ann = xr.DataArray(nan,
                            attrs = daily_ice.attrs,
                            name = 'LAKEICETHICK',
                            dims = ('ensemble', 'lat', 'lon', 'year'),
                            coords = {'ensemble':daily_ice.ensemble,'lat':daily_ice.lat,'lon':daily_ice.lon, 'year':np.arange(1850,2100)})

In [29]:
for i in tqdm(np.arange(0,90), desc = '1st loop'):
    daily_ice_oneens = daily_ice_res[i,:,:,:,:]
    dayofyear_mask = daily_ice_oneens.dayofyear.where(daily_ice_oneens >0)
    iceduration_mod[i,:,:,:] = dayofyear_mask.count('dayofyear')
    iceon_mod[i,:,:,:]       = dayofyear_mask.min('dayofyear')
    iceoff_mod[i,:,:,:]      = dayofyear_mask.max('dayofyear')
    daily_ice_max[i,:,:,:]   = daily_ice_oneens.max('dayofyear')
    ice_thick_ann[i,:,:,:]   = daily_ice_oneens.mean('dayofyear')

1st loop: 100%|██████████| 10/10 [07:50<00:00, 47.08s/it]


In [30]:
'''
we have shifted the time index by -232 days, in order to get the correct day of year, we now shift it back
'''
iceon_mod       = iceon_mod.where(iceduration_mod > 5, nan) + 232
iceoff_mod      = iceoff_mod.where(iceduration_mod > 5, nan)+ 232
iceduration_mod = iceduration_mod.where(iceduration_mod >5,0) 

In [31]:
iceon_mod       = iceon_mod.transpose('ensemble','year','lat','lon')
iceoff_mod      = iceoff_mod.transpose('ensemble','year','lat','lon')
iceduration_mod = iceduration_mod.transpose('ensemble','year','lat','lon')
daily_ice_max   = daily_ice_max.transpose('ensemble','year','lat','lon')
ice_thick_ann   = ice_thick_ann.transpose('ensemble','year','lat','lon')

In [33]:
'''
the Southern Hemisphere is different from the Northern Hemisphere, in which one ice cover season lies in one calendar year
so the calculation is different from the Northern Hemisphere calculation.
there is no shift in time index for the calculation in Southern Hemisphere  
'''
## calculate southern Hemisphere
daily_ice_SH = ice_ds.LAKEICETHICK.sel(lat = slice (-64, -16), lon = slice(276,304), time = slice('1850-01-01','2099-12-31'))
iceon_mod_SH = xr.DataArray(nan, 
                     attrs=daily_ice_SH.attrs,
                     name='LAKEICETHICK',
                     dims=('ensemble','lat','lon','year'), 
                     coords = {'ensemble':daily_ice_SH.ensemble,'lat':daily_ice_SH.lat,'lon':daily_ice_SH.lon,'year':np.arange(1850,2100)})
iceoff_mod_SH = xr.DataArray(nan, 
                     attrs=daily_ice_SH.attrs,
                     name='LAKEICETHICK',
                     dims=('ensemble','lat','lon','year'), 
                     coords = {'ensemble':daily_ice_SH.ensemble,'lat':daily_ice_SH.lat,'lon':daily_ice_SH.lon,'year':np.arange(1850,2100)})
daily_ice_SH_res = daily_ice_SH.assign_coords(time = ind).unstack('time')
for i in tqdm(np.arange(90), desc = '1st loop'):
    daily_ice_oneens = daily_ice_SH_res[i,:,:,:,:]
    dayofyear_mask = daily_ice_oneens.dayofyear.where(daily_ice_oneens >0)
    iceon_mod_SH[i,:,:,:]       = dayofyear_mask.min('dayofyear')
    iceoff_mod_SH[i,:,:,:]      = dayofyear_mask.max('dayofyear')
iceon_mod_SH       = iceon_mod_SH.where(iceon_mod_SH > 5, nan) 
iceoff_mod_SH      = iceoff_mod_SH.where(iceoff_mod_SH > 5, nan)
iceon_mod_SH       = iceon_mod_SH.transpose('ensemble','year','lat','lon')
iceoff_mod_SH      = iceoff_mod_SH.transpose('ensemble','year','lat','lon')

In [34]:
## put the northern Hemisphere and southern Hemsiphere together
iceon_mod[:,:,28:79,221:244] = iceon_mod_SH
iceoff_mod[:,:,28:79,221:244] = iceoff_mod_SH

In [108]:
# write them into nc files
iceduration_mod.rename('iceduration').to_netcdf('/proj/lhuang/LENS/ICEDURATION_mod.nc')
iceon_mod.rename('iceon').to_netcdf('/proj/lhuang/LENS/ICEON_mod.nc')
iceoff_mod.rename('iceoff').to_netcdf('/proj/lhuang/LENS/ICEOFF_mod.nc')
daily_ice_max.rename('icemax').to_netcdf('/proj/lhuang/LENS/ICEMAX_mod.nc')